# ZVP-based reverse-engineering

In [47]:
import io
import sympy
from sympy import FF, sympify, symbols, Poly, Monomial
from collections import Counter
import tabulate
from functools import partial
from itertools import product
from IPython.display import HTML, display
from tqdm.notebook import tqdm
from anytree import RenderTree

from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters, load_params_ecgen
from pyecsca.ec.formula import FormulaAction
from pyecsca.ec.formula.fake import FakeAdditionFormula, FakeDoublingFormula, FakePoint
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod, gcd
from pyecsca.sca.re.tree import build_distinguishing_tree
from pyecsca.sca.re.rpa import MultipleContext
from pyecsca.sca.re.zvp import zvp_points, compute_factor_set
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.mult import LTRMultiplier, AccumulationOrder
from pyecsca.misc.cfg import getconfig
from pyecsca.ec.error import NonInvertibleError
from pyecsca.sca.re.zvp import unroll_formula, compute_factor_set, zvp_points

In [2]:
cfg = getconfig()
cfg.ec.mod_implementation = "gmp"

In [3]:
model = ShortWeierstrassModel()
coordsaff = AffineCoordinateModel(model)
coords = model.coordinates["projective"]
add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
neg = coords.formulas["neg"]

# A 64-bit prime order curve for testing things out
p = 0xc50de883f0e7b167
a = Mod(0x4833d7aa73fa6694, p)
b = Mod(0xa6c44a61c5323f6a, p)
gx = Mod(0x5fd1f7d38d4f2333, p)
gy = Mod(0x21f43957d7e20ceb, p)
n = 0xc50de885003b80eb
h = 1

field = FF(p)

infty = Point(coords, X=Mod(0, p), Y=Mod(1, p), Z=Mod(0, p))
g = Point(coords, X=gx, Y=gy, Z=Mod(1, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a,b=b))
params = DomainParameters(curve, g, n, h)

In [37]:
curves = list(map(lambda spec: load_params_ecgen(io.BytesIO(spec.encode()), "projective"), [
    """[{"field":{"p":"0xa7ec3617d4166b2d"},"a":"0x372994d9d680a83b","b":"0xa0a2bf719d8e68c5","order":"0xa7ec3618be1dab55","subgroups":[{"x":"0x1ef15756946a5b6d","y":"0x2ca9658f7ab9a558","order":"0xa7ec3618be1dab55","cofactor":"0x1","points":[{"x":"0x1ef15756946a5b6d","y":"0x2ca9658f7ab9a558","order":"0xa7ec3618be1dab55"}]}]}]""",
    """[{"field":{"p":"0xa42c1467a1ed04f3"},"a":"0x55d07340a4572f2d","b":"0x0a938c37dfb0b6d5","order":"0xa42c14689284d3a7","subgroups":[{"x":"0x8633981c83ed43a2","y":"0x7b5374e9d7997199","order":"0xa42c14689284d3a7","cofactor":"0x1","points":[{"x":"0x8633981c83ed43a2","y":"0x7b5374e9d7997199","order":"0xa42c14689284d3a7"}]}]}]""",
    """[{"field":{"p":"0xea0d9cead19016ab"},"a":"0xcbbfe501c4ef6d92","b":"0x5762de777a6d9178","order":"0xea0d9cea8cd2c857","subgroups":[{"x":"0xe7daa3e061c3111b","y":"0x56ee59a6845c5e93","order":"0xea0d9cea8cd2c857","cofactor":"0x1","points":[{"x":"0xe7daa3e061c3111b","y":"0x56ee59a6845c5e93","order":"0xea0d9cea8cd2c857"}]}]}]""",
    """[{"field":{"p":"0x9c7e7216decb71a7"},"a":"0x324ef48887401a87","b":"0x3ce6f35a00280102","order":"0x9c7e72175ebfe709","subgroups":[{"x":"0x34683229b405418d","y":"0x308c923cae004514","order":"0x9c7e72175ebfe709","cofactor":"0x1","points":[{"x":"0x34683229b405418d","y":"0x308c923cae004514","order":"0x9c7e72175ebfe709"}]}]}]""",
    """[{"field":{"p":"0xeb5779f0bbf1ef5b"},"a":"0x2419e8bbc7b5f8f2","b":"0xe74e5d3064a4f2e3","order":"0xeb5779f21320c2e9","subgroups":[{"x":"0x3b6c269560abeb00","y":"0x29d157628e75e1c0","order":"0xeb5779f21320c2e9","cofactor":"0x1","points":[{"x":"0x3b6c269560abeb00","y":"0x29d157628e75e1c0","order":"0xeb5779f21320c2e9"}]}]}]""",
    """[{"field":{"p":"0x97b6ea097868b95d"},"a":"0x550a41d65e4bcd13","b":"0x47c5e527113b261c","order":"0x97b6ea094947a76b","subgroups":[{"x":"0x1e669fe19c865bd9","y":"0x05a6bb891920440f","order":"0x97b6ea094947a76b","cofactor":"0x1","points":[{"x":"0x1e669fe19c865bd9","y":"0x05a6bb891920440f","order":"0x97b6ea094947a76b"}]}]}]""",
    """[{"field":{"p":"0xa00629e6522032f7"},"a":"0x896f04a7ae302922","b":"0x6bc03365b1f1cb50","order":"0xa00629e5c03cf913","subgroups":[{"x":"0x14b7b48954936d4e","y":"0x670dc776273bf899","order":"0xa00629e5c03cf913","cofactor":"0x1","points":[{"x":"0x14b7b48954936d4e","y":"0x670dc776273bf899","order":"0xa00629e5c03cf913"}]}]}]""",
    """[{"field":{"p":"0xd47ec1d03a62686d"},"a":"0xd00a3ee0f5c86b02","b":"0x457a5b6c47db38d8","order":"0xd47ec1d107db7d6f","subgroups":[{"x":"0x41ebc3b763f3cd1b","y":"0x3d6925f214620e0c","order":"0xd47ec1d107db7d6f","cofactor":"0x1","points":[{"x":"0x41ebc3b763f3cd1b","y":"0x3d6925f214620e0c","order":"0xd47ec1d107db7d6f"}]}]}]""",
    """[{"field":{"p":"0xb1c9115c6f40d755"},"a":"0x79d3ceefafc44ce9","b":"0x8316af84264df42b","order":"0xb1c9115d17f84a45","subgroups":[{"x":"0x8b0a274089b53fe5","y":"0x3508d33c4beba5ad","order":"0xb1c9115d17f84a45","cofactor":"0x1","points":[{"x":"0x8b0a274089b53fe5","y":"0x3508d33c4beba5ad","order":"0xb1c9115d17f84a45"}]}]}]""",
    """[{"field":{"p":"0x8f738fda18cd5dff"},"a":"0x4747f2f9b8628cbf","b":"0x586cdb9378a1389f","order":"0x8f738fd8fc7ebed3","subgroups":[{"x":"0x7ad306c73b64c1b5","y":"0x69e3ca555190da4b","order":"0x8f738fd8fc7ebed3","cofactor":"0x1","points":[{"x":"0x7ad306c73b64c1b5","y":"0x69e3ca555190da4b","order":"0x8f738fd8fc7ebed3"}]}]}]""",
    """[{"field":{"p":"0xceaf446a53f14bc1"},"a":"0x0000000000000000","b":"0x326539376260f173","order":"0xceaf446aae275419","subgroups":[{"x":"0x98fe44948c3f8678","y":"0x3d440ee959a912d7","order":"0xceaf446aae275419","cofactor":"0x1","points":[{"x":"0x98fe44948c3f8678","y":"0x3d440ee959a912d7","order":"0xceaf446aae275419"}]}]}]""",
    """[{"field":{"p":"0x9d9119957f02fe3f"},"a":"0x0106903196d88df9","b":"0x0000000000000000","order":"0x9d9119957f02fe40","subgroups":[{"x":"0x191a36b9cd81de96","y":"0x10f2c6bded391aa9","order":"0x9d9119957f02fe40","cofactor":"0x1","points":[{"x":"0x0000000000000000","y":"0x0000000000000000","order":"0x2"},{"x":"0x95913fae9065da0f","y":"0x5eeddeee7152d6fb","order":"0x276446655fc0bf9"}]}]}]"""
]))

In [5]:
def simulated_oracle(scalar, affine_point, real_coord_name="projective", real_add_name="add-2007-bl", real_dbl_name="dbl-2007-bl"):
    real_coords = model.coordinates[real_coord_name]
    real_add = real_coords.formulas[real_add_name]
    real_dbl = real_coords.formulas[real_dbl_name]
    real_mult = LTRMultiplier(real_add, real_dbl, None, False, AccumulationOrder.PeqPR, True, True)
    point = affine_point.to_model(params.curve.coordinate_model, params.curve)
    with local(DefaultContext()) as ctx:
        real_mult.init(params, point)
        real_mult.multiply(scalar)

    trace = []

    def callback(action):
        if isinstance(action, FormulaAction):
            for intermediate in action.op_results:
                trace.append(intermediate.value)
    ctx.actions.walk(callback)
    return any(int(value) == 0 for value in trace)

In [6]:
adds = list(filter(lambda formula: formula.name.startswith("add"), coords.formulas.values()))
dbls = list(filter(lambda formula: formula.name.startswith("dbl"), coords.formulas.values()))
formula_pairs = list(product(adds, dbls))

# Compute all of the factor sets, and adjust those for dbl to the right index.
add_fsets = {add: compute_factor_set(add) for add in adds}
dbl_fsets = {}
for dbl in dbls:
    fset = compute_factor_set(dbl)
    dbl2_fset = set()
    for poly in fset:
        pl = poly.copy()
        for symbol in poly.free_symbols:
            original = str(symbol)
            if original.endswith("1"):
                new = original.replace("1", "2")
                pl = pl.subs(original, new)
        dbl2_fset.add(pl)
    dbl_fsets[dbl] = dbl2_fset

In [51]:
scalar = 123456789
bound = 25

chains = []
for i, params in enumerate(curves):
    fake_add = FakeAdditionFormula(params.curve.coordinate_model)
    fake_dbl = FakeDoublingFormula(params.curve.coordinate_model)
    fake_mult = LTRMultiplier(fake_add, fake_dbl, None, False, AccumulationOrder.PeqPR, True, True)
    fake_mult.init(params, FakePoint(params.curve.coordinate_model))
    
    with local(MultipleContext()) as mctx:
        fake_mult.multiply(scalar)

    chain = []

    for point, parents in mctx.parents.items():
        if not parents:
            continue
        k = mctx.points[parents[0]]
        # TODO: look at this k computation
        if len(parents) == 2:
            if k > bound:
                continue
            chain.append(("add", k))
        else:
            chain.append(("dbl", k))
    chains.append(chain)

point_sets = {}
for chain, params in tqdm(zip(chains, curves), desc="Compute ZVP points", total=len(chains)):
    for add in tqdm(adds, desc="Compute ZVP points for adds", leave=False):
        add_fset = add_fsets[add]
        for op, k in tqdm(chain, leave=False):
            if op == "add" and (add, k, params) not in point_sets:
                points = set()
                for poly in add_fset:
                    try:
                        points.update(zvp_points(poly, params.curve, k, params.order))
                    except NonInvertibleError:
                        pass
                point_sets[(add, k, params)] = points
    for dbl in tqdm(dbls, desc="Compute ZVP points for dbls", leave=False):
        dbl_fset = dbl_fsets[dbl]
        for op, k in tqdm(chain, leave=False):
            if op == "dbl" and (dbl, k, params) not in point_sets:
                points = set()
                for poly in dbl_fset:
                    try:
                        points.update(zvp_points(poly, params.curve, k, params.order))
                    except NonInvertibleError:
                        pass
                point_sets[(dbl, k, params)] = points

Compute ZVP points:   0%|          | 0/12 [00:00<?, ?it/s]

Compute ZVP points for adds:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

...

Compute ZVP points for dbls:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

In [63]:
point_map = {}
for add, dbl in formula_pairs:
    points = set()
    for chain, params in zip(chains, curves):
        
        for op, k in chain:
            if op == "add" and (add, k, params) in point_sets:
                point_set = point_sets[(add, k, params)]
            elif op == "dbl" and (dbl, k, params) in point_sets:
                point_set = point_sets[(dbl, k, params)]
            for point in point_set:
                points.add((point, params, k))
    point_map[(add, dbl)] = points
all_points = set().union(*point_map.values())

remapped_point_map = {}
for add, dbl in tqdm(formula_pairs, desc="Remapping"):
    mult = LTRMultiplier(add, dbl, None, False, AccumulationOrder.PeqPR, True, True)
    new_points = set()
    for point, params, k in tqdm(all_points, leave=False):
        mult.init(params, point.to_model(params.curve.coordinate_model, params.curve))
        with local(DefaultContext()) as ctx:
            mult.multiply(scalar)
        trace = []

        def callback(action):
            if isinstance(action, FormulaAction):
                for intermediate in action.op_results:
                    trace.append(intermediate.value)
        # TODO: What is going on? And where is it going on?
        # TODO: Run through all coord systems at once.
        # TODO: Maybe combine with EPA?
        # TODO: Maybe extend oracle with number or position?
        # TODO: k in map? why?
        ctx.actions.walk(callback)
        hit_zero = any(int(value) == 0 for value in trace)
        should_hit_zero = (point, params, k) in point_map[(add, dbl)]
        if hit_zero:
            new_points.add((point, params, k))
        #if hit_zero != should_hit_zero:
        #    print(add, dbl, hit_zero, should_hit_zero, point, params, k)
        #else:
        #    print("ok", hit_zero, should_hit_zero)
    remapped_point_map[(add, dbl)] = new_points

Remapping:   0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

...

  0%|          | 0/2200 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

In [64]:
tree = build_distinguishing_tree(remapped_point_map)

In [65]:
print(RenderTree(tree).by_attr(lambda n: n.name if n.name is not None else n.cfgs))

[x=3034406243747257516, y=11482511777964072902]
DomainParameters(EllipticCurve)
6
├── [x=6126421359781338627, y=9772906831073546368]
│   DomainParameters(EllipticCurve)
│   1
│   ├── [x=9219828953749013321, y=2141216720201514772]
│   │   DomainParameters(EllipticCurve)
│   │   3858024
│   │   ├── [x=5047606674150175385, y=1956997826490032405]
│   │   │   DomainParameters(EllipticCurve)
│   │   │   6
│   │   │   ├── (AdditionEFDFormula(add-2007-bl for shortw/projective), DoublingEFDFormula(dbl-2015-rcb for shortw/projective))
│   │   │   │   (AdditionEFDFormula(add-2002-bj for shortw/projective), DoublingEFDFormula(dbl-2015-rcb for shortw/projective))
│   │   │   └── (AdditionEFDFormula(add-2015-rcb for shortw/projective), DoublingEFDFormula(dbl-2015-rcb for shortw/projective))
│   │   └── (AdditionEFDFormula(add-2015-rcb for shortw/projective), DoublingEFDFormula(dbl-2007-bl for shortw/projective))
│   │       (AdditionEFDFormula(add-2015-rcb for shortw/projective), DoublingEFDFormula(

In [77]:
for pair in point_map.keys():
    print(pair[0], pair[1],
          len(point_map[pair]),
          len(remapped_point_map[pair]),
          -len(point_map[pair].difference(remapped_point_map[pair])),
          len(remapped_point_map[pair].difference(point_map[pair])))
    print(point_map[pair].difference(remapped_point_map[pair]))

shortw/projective/add-2007-bl shortw/projective/dbl-2007-bl 462 518 0 56
set()
shortw/projective/add-2007-bl shortw/projective/dbl-1998-cmo 462 518 0 56
set()
shortw/projective/add-2007-bl shortw/projective/dbl-2015-rcb 1716 1784 0 68
set()
shortw/projective/add-2007-bl shortw/projective/dbl-1998-cmo-2 462 518 0 56
set()
shortw/projective/add-2015-rcb shortw/projective/dbl-2007-bl 850 870 0 20
set()
shortw/projective/add-2015-rcb shortw/projective/dbl-1998-cmo 850 870 0 20
set()
shortw/projective/add-2015-rcb shortw/projective/dbl-2015-rcb 2104 2108 0 4
set()
shortw/projective/add-2015-rcb shortw/projective/dbl-1998-cmo-2 850 870 0 20
set()
shortw/projective/add-2002-bj shortw/projective/dbl-2007-bl 462 518 0 56
set()
shortw/projective/add-2002-bj shortw/projective/dbl-1998-cmo 462 518 0 56
set()
shortw/projective/add-2002-bj shortw/projective/dbl-2015-rcb 1716 1784 0 68
set()
shortw/projective/add-2002-bj shortw/projective/dbl-1998-cmo-2 462 518 0 56
set()
shortw/projective/add-1998-c